# DEEPX Tutorial 05 - DX Compile workflow


In Tutorial 5, you will practice compiling classification, object detection, and segmentation models using the DX-Compiler. We will also review the guide for troubleshooting problems during compilation.

For more details, refer to the DX-COM user guide 👉 [Download](https://developer.deepx.ai/?files=MjUxNQ==)

>This tutorial is based on dx-all-suite v2.0.0, released in September 2025.

## Compiling Image Classification Model (MobileNetV2)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [ ]:
# Install pytorch & onnx
!pip install --quiet torch torchvision onnx

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

Export Pytorch based MobileNetV2 model to ONNX

In [ ]:
import torch, torchvision

# Load MobileNetV2 model
model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.eval()

# Batch size must be 1
dummy_input = torch.randn(1, 3, 224, 224)

onnx_path = "MobilenetV2.onnx"

torch.onnx.export(
    model,                      # PyTorch model object to export
    dummy_input,                # Dummy input used for tracing (tuple is possible)
    onnx_path,                  # Output ONNX file path
    export_params=True,         # If True, saves model parameter (weight) into the ONNX file
    input_names=["input_test"], # Name of the ONNX model input tensor
    output_names=["output"],    # Name of the ONNX model output tensor
    opset_version=13            # ONNX opset version (recommended: 11 ~ 21)
)
print("✅ Save ONNX:", onnx_path)

### 2. JSON configuration for Input/Pre-processing/Calibration

Generate a configuration file for Input/Pre-processing/Calibration of MobilenetV2.

This JSON configuration file incudes:
 - Input specifications
 - Calibration methods
 - Data preprocessing settings
 - Optional parameters for advanced compilation schemes

Model **Input Restrictions**:
 - The batch size must be fixed to 1
 - Only a single input is supported (Multi-input will be supported in 2026)
 - Input name must exactly match ONNX model definition

#### 2.1. Incorrect input name case

This MobilenetV2 model's input name is `input_test` as shown below:

![](assets/mobilenetv2-input-name.png)

Input name (`input_test` in this example) must exactly match the input name of JSON configuration.

However, you can see there is wrong input name `incorrect_input_name` in the following JSON configuration for inputs.

In [14]:
%%writefile MobilenetV2.json
{
  "inputs": {"incorrect_input_name": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Overwriting MobilenetV2.json


Let's run DX-COM with this wrong JSON configuration. You will meet ERROR:
> ConfigInputError: The input name in config incorrect_input_name is not same as model input input_test

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 2.2. Incorrect input shape
In the following JSON file, the input name has the correct one - `input_test`.

However, even if your input shape is 1x3x224x224 (BxCxHxW), calibration image shape has 224x224x3 (HxWxC). You must change the shape of calibraiton image to match with 1x3x224x224 (BxCxHxW) by using `transpose` and `expandDim`.

In [15]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Overwriting MobilenetV2.json


Let's run DX-COM with this wrong JSON configuration. You will meet ERROR:

> ConfigInputError: Config shape [1, 3, 224, 224] does not match preprocessed data shape [1, 224, 224, 3]

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 2.3. Add `transpose` & `expanDim` to JSON

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

### 3. Compile with DX-Compiler and verify .dxnn
Compile with dx_com to generate .dxnn

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

In [ ]:
!run_model -m MobilenetV2.dxnn

## Compiling Object Detection Model (YOLOv9s)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

#### 1.1. Download `yolov9-s.pt` pytorch model to your local system:

In [ ]:
#!cd yolov9 && pip install -r requirements.txt
!pip install torch==2.5.1 torchvision==0.20.1
!pip install onnx onnxsim

In [ ]:
!wget --no-check-certificate https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt

#### 1.2. Export the downloaded torch model to ONNX:

Download yolov9 git repo to use `export.py` for yolov9.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git

Export Pytorch based `yolov9-s` model to ONNX

In [ ]:
!cd yolov9 && python3 export.py --weights ../yolov9-s.pt \
                                --img-size 640 640 \
                                --opset 12 \
                                --simplify \
                                --batch-size 1 \
                                --include onnx

Check if `yolov9-s.onnx` file is generated.

In [ ]:
!ls | grep yolov9-s.onnx

### 2. JSON configuration for Input/Pre-processing/Calibration

In [13]:
%%writefile yolov9-s.json
{
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

Overwriting abc.json


### 3. Compile with DX-Compiler and verify .dxnn

In [ ]:
!./dx_com/dx_com -m yolov9-s.onnx -c yolov9-s.json -o ./

Check if **yolov9-s.dxnn** file is generated as expected:

In [ ]:
!ls | grep yolov9-s.dxnn

In [ ]:
!run_model -m yolov9-s.dxnn

## Compiling ViT Model

https://github.com/mlfoundations/open_clip
